In [1]:
!pip install -q huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 33.7 MB/s eta 0:00:00:00:0100:01


In [2]:
from datasets import load_dataset

daa_file_path='/kaggle/input/building-the-fine-tuning-dataset/train.csv'

train_ds=load_dataset('csv', data_files=daa_file_path)['train']
train_ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 4984
})

In [3]:
train_ds[0]

{'anchor': '<think>\nLet’s think through this step by step\nApril: 48 clips\nMay: 48 / 2 = 24 clips\nTotal: 48 + 24 = 72 clips\n</think>\n\\boxed{72}',
 'positive': '<think>\nLet’s think through this step by step\nApril: 48 clips\nMay: 48 / 2 = 24 clips\nTotal = 48 + 24 = 72 clips\n</think>\n\\boxed{72}'}

# Pre-processing dataset

We will split dataset into training, testing and evaluation.

In [4]:
train_test_split=train_ds.train_test_split(test_size=0.1, seed=42)
train_test_split

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 4485
    })
    test: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 499
    })
})

In [5]:
train_eval_split=train_test_split['train'].train_test_split(test_size=0.1, seed=42)
train_eval_split

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 4036
    })
    test: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 449
    })
})

In [6]:
from datasets import DatasetDict

findal_ds=DatasetDict({
    'train': train_eval_split['train'],
    'test': train_eval_split['test'],
    'eval': train_test_split['test']
})
findal_ds

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 4036
    })
    test: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 449
    })
    eval: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 499
    })
})

In [7]:
MODEL_NAME='all-MiniLM-L6-v2'
OUTPUT_DIR='encoder-L6-V2/training'
MODEL_DIR='aisuko/encoder-L6-V2'

# Load the Model

In [8]:
# https://www.sbert.net/docs/package_reference/sentence_transformer/losses.html#multiplenegativesrankingloss
import torch
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=SentenceTransformer(MODEL_NAME).to(device)
model

2025-05-01 10:24:18.021038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746095058.200499      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746095058.252326      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [9]:
from sentence_transformers.losses import MultipleNegativesRankingLoss

loss=MultipleNegativesRankingLoss(model)

In [10]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)

README.md:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/471k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/142k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/108k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [11]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers import SentenceTransformerTrainer
from transformers import EarlyStoppingCallback


train_args=SentenceTransformerTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    report_to='tensorboard',
    run_name='encoder-L6-V2'
)

# evaluate the base model
dev_evaluator(model)

trainer=SentenceTransformerTrainer(
    model=model,
    args=train_args,
    train_dataset=findal_ds['train'],
    eval_dataset=findal_ds['test'],
    loss=loss,
    evaluator=dev_evaluator,
    
)

# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

# trainer.train(callbacks=[early_stopping_callback])

trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
100,0.017900,0.001918,0.876711,0.874413
200,0.006400,0.004451,0.877158,0.875231
300,0.002300,0.002632,0.877531,0.875441
400,0.001200,0.002503,0.877640,0.875472
500,0.001900,0.002043,0.877565,0.875439


TrainOutput(global_step=506, training_loss=0.005863229480476336, metrics={'train_runtime': 53.4311, 'train_samples_per_second': 151.073, 'train_steps_per_second': 9.47, 'total_flos': 0.0, 'train_loss': 0.005863229480476336, 'epoch': 2.0})

In [12]:
# evaluate the trained model on the test set
test_dataset = load_dataset("sentence-transformers/stsb", split="test")

test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-test",
)

test_evaluator(model)

{'sts-test_pearson_cosine': 0.8775619106040393,
 'sts-test_spearman_cosine': 0.875417425174265}

In [13]:
model.save_pretrained(MODEL_DIR)

In [14]:
import kagglehub

VARIATION_SLUG='v0.1.4'

handler = f'aisuko/encoder-L6-V2/transformers/{VARIATION_SLUG}'

kagglehub.model_upload(
    handle=handler,
    local_model_dir=MODEL_DIR,
    license_name='MIT',
    version_notes='FT with reasoning paths'
)

Uploading Model https://www.kaggle.com/models/aisuko/encoder-L6-V2/transformers/v0.1.4 ...
Starting upload for file aisuko/encoder-L6-V2/special_tokens_map.json


Uploading: 100%|██████████| 695/695 [00:00<00:00, 1.15kB/s]

Upload successful: aisuko/encoder-L6-V2/special_tokens_map.json (695B)
Starting upload for file aisuko/encoder-L6-V2/model.safetensors


Uploading: 100%|██████████| 90.9M/90.9M [00:01<00:00, 60.6MB/s]

Upload successful: aisuko/encoder-L6-V2/model.safetensors (87MB)
Starting upload for file aisuko/encoder-L6-V2/README.md


...

Uploading: 100%|██████████| 349/349 [00:00<00:00, 591B/s]

Upload successful: aisuko/encoder-L6-V2/modules.json (349B)
Starting upload for file aisuko/encoder-L6-V2/1_Pooling/config.json


Uploading: 100%|██████████| 296/296 [00:00<00:00, 492B/s]

Upload successful: aisuko/encoder-L6-V2/1_Pooling/config.json (296B)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/aisuko/encoder-L6-V2/transformers/v0.1.4
